In [ ]:
# SETUP AUTOMATICO E DOWNLOAD DEL PROGETTO DA GITHUB
import os

# 1. Clona il repository (inserisci il tuo vero link GitHub)
!git clone https://github.com/TUO_NOME_UTENTE/AI-Studio.git

# 2. Sposta l'ambiente di lavoro dentro la cartella del progetto
%cd AI-Studio

# 3. Installa le librerie leggendo il file requirements.txt
!pip install -r requirements.txt
!apt-get install -y ffmpeg

print("✅ Ambiente clonato e pronto all'uso! Tutto il codice è sincronizzato.")

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import os
import shutil

# Metadati base
api_key_input = widgets.Password(description="API Key:", style={'description_width': 'initial'})
materia_input = widgets.Text(description="Materia:")
argomento_input = widgets.Text(description="Argomento:")
professore_input = widgets.Text(description="Professore:")

print("⚙️ COMPILA I METADATI E INSERISCI LA CHIAVE API:")
display(api_key_input, materia_input, argomento_input, professore_input)

# Nuova sezione URL
print("\n🔗 OPZIONE A: Scarica da URL (Portale Universitario)")
video_url_input = widgets.Text(description="Link Video:", placeholder="Incolla l'URL qui...", layout=widgets.Layout(width='80%'))
display(video_url_input)
print("Se usi l'URL, carica qui sotto il file cookies.txt:")
cookie_upload = files.upload()

print("\n📤 OPZIONE B: Carica file MP4 dal PC (Ignora se hai usato l'URL)")
video_upload = files.upload()

# Creazione directory organizzate
os.makedirs("data/videos", exist_ok=True)
os.makedirs("data/transcripts", exist_ok=True)

# Salvataggio video caricato nella directory corretta
video_filename = None
if video_upload:
    uploaded_name = list(video_upload.keys())[0]
    video_filename = os.path.join("data/videos", uploaded_name)
    shutil.move(uploaded_name, video_filename)
    print(f"📹 Video salvato in: {video_filename}")

cookie_filename = list(cookie_upload.keys())[0] if cookie_upload else None

print("\n📚 (OPZIONALE) CARICA GLI APPUNTI PRECEDENTI (.tex):")
notes_upload = files.upload()
history_filename = list(notes_upload.keys())[0] if notes_upload else None

In [ ]:
import sys
import os
import warnings

# Silenzia i warning di HuggingFace
warnings.filterwarnings('ignore', category=UserWarning, module='huggingface_hub')
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

# SETTA API KEY PRIMA DI IMPORTARE I MODULI (il config_loader è un Singleton!)
if not api_key_input.value:
    print("❌ ERRORE: API Key mancante! Inseriscila nella cella precedente.")
else:
    os.environ['GOOGLE_API_KEY'] = api_key_input.value

sys.path.append('./src')

# IMPORTA DOPO aver settato la variabile d'ambiente
from audio_processing import process_video_audio
from vision_processing import extract_keyframes
from llm_gen_engine import generate_notes
from google.colab import files
import yt_dlp

# LOGICA DI DOWNLOAD SE URL FORNITO
if video_url_input.value:
    video_name = f"{materia_input.value}_{argomento_input.value}".replace(" ", "_") or "lezione"
    video_filename = os.path.join("data/videos", f"{video_name}.mp4")
    
    ydl_opts = {
        'outtmpl': video_filename,
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    }
    if cookie_filename:
        ydl_opts['cookiefile'] = cookie_filename
        
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url_input.value])

# VALIDAZIONE
if not video_filename:
    print("❌ Carica un video (Cella 2)")
elif not api_key_input.value:
    print("❌ Inserisci API Key (Cella 2)")
else:
    print("🎙️ Fase 1: Trascrizione audio...")
    transcript, transcript_path = process_video_audio(
        video_filename, 
        save_transcript=True,
        transcript_output_dir="data/transcripts"
    )
    
    print("👁️ Fase 2: Estrazione keyframe...")
    keyframes = extract_keyframes(video_filename, output_dir="data/frames")
    
    print("🧠 Fase 3: Generazione LaTeX...")
    latex_output = generate_notes(transcript=transcript, keyframes=keyframes)
    
    nome_file = f"{materia_input.value}_{argomento_input.value}_{professore_input.value}".replace(" ", "_") + ".tex"
    
    with open(nome_file, "w", encoding="utf-8") as f:
        f.write(latex_output)
        
    files.download(nome_file)
    
    print(f"\n✅ Completato! File: {nome_file}")

In [ ]:
from interactive_agent import run_editor_agent, run_tutor_agent
import ipywidgets as widgets
from IPython.display import display, HTML

# Uniamo tutta la trascrizione in un'unica stringa per dare il contesto
full_transcript = " ".join([seg['text'] for seg in transcript])

mode_selector = widgets.Dropdown(
    options=['Modalità TUTOR (Domande)', 'Modalità EDITOR (Modifica LaTeX)'],
    description='Azione:',
    style={'description_width': 'initial'}
)
text_input = widgets.Textarea(description="Richiesta:", layout=widgets.Layout(width='80%', height='100px'))
latex_input = widgets.Textarea(description="Codice da modificare (solo per Editor):", layout=widgets.Layout(width='80%', height='100px'))
submit_button = widgets.Button(description="Invia all'IA", button_style='success')
output_area = widgets.Output()

def on_submit(b):
    with output_area:
        output_area.clear_output()
        print("Elaborazione in corso...")
        if "TUTOR" in mode_selector.value:
            risposta = run_tutor_agent(text_input.value, full_transcript, api_key_input.value)
            display(HTML(f"<b>👨‍🏫 Il Prof risponde:</b><br><pre style='white-space: pre-wrap;'>{risposta}</pre>"))
        else:
            nuovo_latex = run_editor_agent(latex_input.value, text_input.value, full_transcript, api_key_input.value)
            display(HTML(f"<b>📝 Nuovo Codice LaTeX:</b><br><pre>{nuovo_latex}</pre>"))

submit_button.on_click(on_submit)
display(mode_selector, text_input, latex_input, submit_button, output_area)